In [22]:
import sounddevice as sd
import queue
import json
from vosk import Model, KaldiRecognizer



In [23]:
# -------- CONFIG --------
MODEL_PATH = "/Users/olixstudios/Documents/workspace/Projects/vosk-demo/models/vosk-model-small-en-us-0.15"  # change if needed
SAMPLE_RATE = 16000
# ------------------------


def main():
    print("Loading model...")
    model = Model(MODEL_PATH)
    recognizer = KaldiRecognizer(model, SAMPLE_RATE)

    audio_queue = queue.Queue()

    def audio_callback(indata, frames, time, status):
        if status:
            print("Status:", status)
    
        # DEBUG: show that audio bytes are arriving
        # print("Audio chunk size:", len(indata))
    
        audio_queue.put(bytes(indata))

    print("Starting microphone... Speak into the mic. Ctrl+C to stop.")
    with sd.RawInputStream(
        samplerate=SAMPLE_RATE,
        blocksize=8000,
        dtype="int16",
        channels=1,
        callback=audio_callback,
        device=4,
    ):
        while True:
            data = audio_queue.get()
            if recognizer.AcceptWaveform(data):
                result = recognizer.Result()
                text = json.loads(result).get("text", "")
                if text.strip():
                    print("Final:", text)
            else:
                # Partial results while you're speaking
                partial = recognizer.PartialResult()
                partial_text = json.loads(partial).get("partial", "")
                if partial_text.strip():
                    print("Partial:", partial_text, end="\r")



In [24]:
def audio_callback(indata, frames, time, status):
    if status:
        print("Status:", status)

    # DEBUG: show that audio bytes are arriving
    print("Audio chunk size:", len(indata))

    audio_queue.put(bytes(indata))

In [25]:
main()

Loading model...


LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from /Users/olixstudios/Documents/workspace/Projects/vosk-demo/models/vosk-model-small-en-us-0.15/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from /Users/olixstudios/Documents/workspace/Projects/vosk-demo/models/vosk-model-small-en-us-0.15/graph/HCLr.fst /Users/olixstudios/Documents/workspace/Projects/vosk-demo/models/vosk-model-small-en-us-0.15/graph/Gr.fst
LOG (VoskAPI:ReadData

Starting microphone... Speak into the mic. Ctrl+C to stop.
Final: hello hello hello hellolo
Final: yes you can see as you can get itit
Final: can you hear meme
Final: can you hear meme
Final: why does this mean be going meme
Final: how what are you in it listeningng
Final: oh yeah


KeyboardInterrupt: 